<a id='0.1'></a>
## 0.1 load required packages

In [1]:
%run "..\..\..\Startup_py3.py"
sys.path.append(r"..\..\..\..\..\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

# library design specific tools
from ImageAnalysis3.library_tools import LibraryDesigner as ld
from ImageAnalysis3.library_tools import LibraryTools as lt
# biopython imports
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML

15160


In [11]:
import pandas as pd

<a id='0.2'></a>
## 0.2 folders

In [2]:
## Some folders
# human genome
reference_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl'
genome_folder = os.path.join(reference_folder, 'Genome')
# Library directories
pool_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-11_brain'
# folder for sub-pool
library_folder = os.path.join(pool_folder, f'mouse_genome_1000')

In [3]:
# laod encoding
# summarize total readout usage
encoding_folder = os.path.join(pool_folder, f'Encoding_design')
region_2_bits = pickle.load(open(os.path.join(encoding_folder, 'region_2_bits.pkl'), 'rb'))

In [5]:
# load probes
probes = []
final_pb_filename = os.path.join(pool_folder, 'final_pool_probes.fasta')

with open(final_pb_filename, 'r') as handle:
    for record in SeqIO.parse(handle, "fasta"):
        probes.append(record)

In [6]:
# load region names

region_2_names = {}

for _pb in probes:
    _reg = _pb.id.split('loc_')[1].split('_')[0]
    _gene = _pb.id.split('gene_')[1].split('_')[0]
    #print(_reg,_gene)
    
    if _gene not in region_2_names:
        region_2_names[_gene] = _reg
print(region_2_names)

{'1': '1:3740000-3760000', '3': '1:8740000-8760000', '5': '1:13740000-13760000', '7': '1:18740000-18760000', '9': '1:23740000-23760000', '11': '1:28740000-28760000', '13': '1:33740000-33760000', '15': '1:38740000-38760000', '17': '1:43740000-43760000', '19': '1:48740000-48760000', '21': '1:53740000-53760000', '23': '1:58740000-58760000', '25': '1:63740000-63760000', '27': '1:68740000-68760000', '29': '1:73740000-73760000', '31': '1:78740000-78760000', '33': '1:83740000-83760000', '35': '1:88740000-88760000', '37': '1:93740000-93760000', '39': '1:98740000-98760000', '41': '1:103740000-103760000', '43': '1:108740000-108760000', '45': '1:113740000-113760000', '47': '1:118740000-118760000', '49': '1:123740000-123760000', '51': '1:128740000-128760000', '53': '1:133740000-133760000', '55': '1:138740000-138760000', '57': '1:143740000-143760000', '59': '1:148740000-148760000', '61': '1:153740000-153760000', '63': '1:158740000-158760000', '65': '1:163740000-163760000', '67': '1:168740000-168760

In [7]:
region_2_bits[1], region_2_names['1']

(array([36, 62, 77]), '1:3740000-3760000')

In [8]:
# laod encoding
# summarize total readout usage
region_2_readout_dict = pickle.load(open(os.path.join(library_folder, 'region_2_readout.pkl'), 'rb'))
print(region_2_readout_dict)

{1: ['c36', 'c62', 'c77'], 2: ['c17', 'c42', 'c86'], 3: ['c34', 'c72', 'c82'], 4: ['c57', 'c58', 'c94'], 5: ['c41', 'c78', 'c93'], 6: ['c61', 'c65', 'c80'], 7: ['c66', 'c74', 'c83'], 8: ['c23', 'c44', 'c54'], 9: ['c4', 'c45', 'c90'], 10: ['c51', 'c64', 'c75'], 11: ['c12', 'c67', 'c69'], 12: ['c15', 'c43', 'c68'], 13: ['c30', 'c31', 'c62'], 14: ['c10', 'c22', 'c47'], 15: ['c5', 'c6', 'c13'], 16: ['c3', 'c35', 'c49'], 17: ['c39', 'c84', 'c88'], 18: ['c8', 'c20', 'c56'], 19: ['c24', 'c32', 'c55'], 20: ['c18', 'c29', 'c73'], 21: ['c53', 'c89', 'c91'], 22: ['c7', 'c14', 'c63'], 23: ['c0', 'c37', 'c81'], 24: ['c70', 'c95', 'c97'], 25: ['c26', 'c28', 'c60'], 26: ['c11', 'c68', 'c74'], 27: ['c21', 'c56', 'c85'], 28: ['c54', 'c83', 'c87'], 29: ['c2', 'c76', 'c79'], 30: ['c48', 'c51', 'c96'], 31: ['c8', 'c44', 'c65'], 32: ['c1', 'c19', 'c53'], 33: ['c5', 'c27', 'c33'], 34: ['c67', 'c92', 'c98'], 35: ['c10', 'c52', 'c58'], 36: ['c12', 'c34', 'c42'], 37: ['c4', 'c38', 'c77'], 38: ['c40', 'c59', 'c

In [9]:
# load used readouts
readout_usage_file = os.path.join(library_folder, 'readout_usage.pkl')
readout_dict = pickle.load(open(readout_usage_file, 'rb'))

In [10]:
readout_dict

{'c': [SeqRecord(seq=Seq('AACCACGGGCTAGAAATATCCGAGTGGCAT'), id='NDB_784', name='NDB_784', description='NDB_784', dbxrefs=[]),
  SeqRecord(seq=Seq('GCCTAAGACGTGCGACATGTTATGCTAGGC'), id='NDB_755', name='NDB_755', description='NDB_755', dbxrefs=[]),
  SeqRecord(seq=Seq('GCCGTTTCGTATATGTCGATGTCCTTAGAC'), id='NDB_826', name='NDB_826', description='NDB_826', dbxrefs=[]),
  SeqRecord(seq=Seq('TACAAATACCGGTGGTTGTCACTTTCCGAA'), id='NDB_713', name='NDB_713', description='NDB_713', dbxrefs=[]),
  SeqRecord(seq=Seq('TCCGTGACGAGATCCCGGTCGGATATGTGA'), id='NDB_865', name='NDB_865', description='NDB_865', dbxrefs=[]),
  SeqRecord(seq=Seq('GAGCTTATGACTACCACGGCATCGATTTAG'), id='NDB_725', name='NDB_725', description='NDB_725', dbxrefs=[]),
  SeqRecord(seq=Seq('CGCGAAGATCGAACTCGTTGACTAAGCATT'), id='NDB_817', name='NDB_817', description='NDB_817', dbxrefs=[]),
  SeqRecord(seq=Seq('GTGGTAAACCTCAACTACTAAGCCGCGGTA'), id='NDB_710', name='NDB_710', description='NDB_710', dbxrefs=[]),
  SeqRecord(seq=Seq('CCAGTA

## bit_2_readout

In [23]:
save_folder = os.path.join(pool_folder, 'Summary_tables')
dtype_dict = {_v:_k for _k,_v in ia.classes._allowed_kwds.items()}

In [26]:
for _dtype, _records in readout_dict.items():
    
    bit_readout_df = pd.DataFrame([[_i+1, _r.id] for _i, _r in enumerate(_records)], columns=['Bit','ReadoutName'])
    save_filename = os.path.join(save_folder, f'CTP11-mouse-genome-1000_{dtype_dict[_dtype]}_readouts.csv')
    print(save_filename)
    bit_readout_df.to_csv(save_filename, index=False)

\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-11_brain\Summary_tables\CTP11-mouse-genome-1000_combo_readouts.csv


In [25]:
bit_readout_df

,Bit,ReadoutName
0,1,NDB_784
1,2,NDB_755
2,3,NDB_826
3,4,NDB_713
4,5,NDB_865
...,...,...
94,95,NDB_940
95,96,NDB_872
96,97,NDB_934
97,98,NDB_926


# Codebook

In [13]:
# generate the codebook
codebook = pd.DataFrame(columns=['name','id']+[_r.id for _r in readout_dict['c']])
#codebook.add(['name', 'id'], axis=1)
codebook['id'] = list(region_2_readout_dict.keys())

In [14]:
codebook['name'] = [region_2_names[str(_r)] for _r in list(region_2_readout_dict.keys())]

In [15]:
codebook

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_1028,NDB_925,NDB_824,NDB_712,NDB_818,NDB_940,NDB_872,NDB_934,NDB_926,NDB_910
0,1:3740000-3760000,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1:6240000-6260000,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1:8740000-8760000,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1:11240000-11260000,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1:13740000-13760000,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983,19:53740000-53760000,989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
984,19:56240000-56260000,990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
985,19:58740000-58760000,991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
986,19:61240000-61260000,992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
max_bit = np.max([np.max([int(_b[1:]) for _b in _bits]) for _gname, _bits in region_2_readout_dict.items()]) + 1

In [17]:
from tqdm.notebook import tqdm
for _gname, _bits in tqdm(region_2_readout_dict.items()):
    binary_code = []
    for _i in range(max_bit):
        if f"c{_i}" in _bits:
            binary_code.append(1)
        else:
            binary_code.append(0)
    #print(_gname)
    codebook.loc[codebook['id']==_gname, codebook.columns[2:]] = binary_code
    #codebook.loc[codebook['name']==_gname,'id'] = int(_bits[0].split('u')[1])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
140
141
142
143
144
145
146
147
148
150
151
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
190
191
192
193
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
266
267
268
269
270
271
272
273
274
276
277
278
279
281
282
283
285
286
287
288
289
290
29

In [18]:
from tqdm.notebook import tqdm

In [20]:
chr_2_region_num = pickle.load(open(os.path.join(library_folder, 'chr_2_region_num.pkl'), 'rb'))
chr_2_region_num

{'1': 77,
 '2': 70,
 '3': 61,
 '4': 59,
 '5': 58,
 '6': 58,
 '7': 49,
 '8': 50,
 '9': 48,
 '10': 51,
 '11': 48,
 '12': 43,
 '13': 45,
 '14': 42,
 '15': 40,
 '16': 38,
 '17': 36,
 '18': 35,
 '19': 23,
 'X': 56,
 'Y': 1}

In [21]:
chr_2_region_names = pickle.load(open(os.path.join(library_folder, 'chr_2_region_names.pkl'), 'rb'))
chr_2_region_names

{'1': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77],
 '2': [80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  150,
  151],
 '3': [153,
  154,
  155,
  156,
  157,
  158,
  159,
  160,
  16

In [22]:
# generate region_to_chr_id
region_2_chr = {}
region_2_chr_order = {}
for _chr, _regions in chr_2_region_names.items():
    for _i, _region in enumerate(_regions):
        region_2_chr[_region] = _chr
        region_2_chr_order[_region] = _i

In [23]:
codebook['chr'] = [region_2_chr[_g] for _g in codebook['id']]
codebook['chr_order'] = [region_2_chr_order[_g] for _g in codebook['id']]

In [24]:
codebook.sort_values('id', inplace=True)

In [25]:
save_folder = os.path.join(pool_folder, 'Summary_tables')
save_filename = os.path.join(save_folder, 'CTP11-mouse-genome-1000_codebook.csv')

In [26]:
codebook.to_csv(save_filename,
                index=None)

# generate 500 gene codebook

In [27]:
codebook_500 = pd.DataFrame(columns=codebook.columns)

# extract 500 genes
sel_regions_500 = []
for _pb in probes:
    if _pb.id.split('library_')[-1] == '1000-mouse-genome_500':
        _gene = _pb.id.split('gene_')[1].split('_')[0]
        if _gene not in sel_regions_500:
            sel_regions_500.append(_gene)
            
for _gene in sel_regions_500:
    codebook_500 = codebook_500.append(codebook.loc[codebook['id']==int(_gene)])
codebook_500.sort_values('id', inplace=True)
codebook_500

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_824,NDB_712,NDB_818,NDB_940,NDB_872,NDB_934,NDB_926,NDB_910,chr,chr_order
0,1:3740000-3760000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1:8740000-8760000,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2
4,1:13740000-13760000,5,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,4
6,1:18740000-18760000,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,6
8,1:23740000-23760000,9,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,X:151240000-151260000,1053,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,48
433,X:156240000-156260000,1055,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,50
435,X:161240000-161260000,1057,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,X,52
437,X:166240000-166260000,1059,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,X,54


In [28]:
save_folder = os.path.join(pool_folder, 'Summary_tables')
save_filename = os.path.join(save_folder, 'CTP11-mouse-genome-500_codebook.csv')

codebook_500.to_csv(save_filename, index=None)

# 250-1

In [81]:
codebook_250_1 = pd.DataFrame(columns=codebook.columns)

# extract 250_1 genes
sel_regions_250_1 = []
for _pb in probes:
    if _pb.id.split('library_')[-1] == '1000-mouse-genome_250_1':
        _gene = _pb.id.split('gene_')[1].split('_')[0]
        if _gene not in sel_regions_250_1:
            sel_regions_250_1.append(_gene)
            
for _gene in sel_regions_250_1:
    codebook_250_1 = codebook_250_1.append(codebook.loc[codebook['id']==int(_gene)])
codebook_250_1.sort_values('id', inplace=True)
codebook_250_1

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_824,NDB_712,NDB_818,NDB_940,NDB_872,NDB_934,NDB_926,NDB_910,chr,chr_order
1,1:6240000-6260000,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
5,1:16240000-16260000,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,5
9,1:26240000-26260000,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,9
13,1:36240000-36260000,14,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,13
17,1:46240000-46260000,18,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,X:116240000-116260000,1039,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,37
424,X:131240000-131260000,1045,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,41
428,X:141240000-141260000,1049,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,45
432,X:153740000-153760000,1054,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,49


In [82]:
save_folder = os.path.join(pool_folder, 'Summary_tables')
save_filename = os.path.join(save_folder, 'CTP11-mouse-genome-250_1_codebook.csv')

codebook_250_1.to_csv(save_filename, index=None)

# 250-2

In [83]:
codebook_250_2 = pd.DataFrame(columns=codebook.columns)

# extract 250_2 genes
sel_regions_250_2 = []
for _pb in probes:
    if _pb.id.split('library_')[-1] == '1000-mouse-genome_250_2':
        _gene = _pb.id.split('gene_')[1].split('_')[0]
        if _gene not in sel_regions_250_2:
            sel_regions_250_2.append(_gene)
            
for _gene in sel_regions_250_2:
    codebook_250_2 = codebook_250_2.append(codebook.loc[codebook['id']==int(_gene)])
codebook_250_2.sort_values('id', inplace=True)
codebook_250_2

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_824,NDB_712,NDB_818,NDB_940,NDB_872,NDB_934,NDB_926,NDB_910,chr,chr_order
3,1:11240000-11260000,4,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,3
7,1:21240000-21260000,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,7
11,1:31240000-31260000,12,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,11
15,1:41240000-41260000,16,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,15
19,1:51240000-51260000,20,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,X:121240000-121260000,1041,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,X,39
426,X:136240000-136260000,1047,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,43
430,X:146240000-146260000,1051,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,X,47
434,X:158740000-158760000,1056,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,51


In [84]:
save_folder = os.path.join(pool_folder, 'Summary_tables')
save_filename = os.path.join(save_folder, 'CTP11-mouse-genome-250_2_codebook.csv')

codebook_250_2.to_csv(save_filename, index=None)